# Yelp Scraper - Reviews

Scrape Yelp reviews using a url.

In [237]:
import requests
import lxml.html
import pandas as pd
import re
import time
from selenium import webdriver
from bs4 import BeautifulSoup

### Load Page

In [238]:
chrome_path = r"/Users/selenalu/Documents/chromedriver"
driver = webdriver.Chrome(chrome_path)

In [239]:
url = input('url: ')

url: https://www.yelp.com/biz/waba-grill-los-angeles-24


In [240]:
driver.get(url)

### Scrape Information

Get name.

In [241]:
element = driver.find_elements_by_xpath('//h1[@class="lemon--h1__373c0__2ZHSL heading--h1__373c0__dvYgw undefined heading--inline__373c0__10ozy"]')
name = element[0].text

Get star rating.

In [242]:
html = driver.page_source 
soup = BeautifulSoup(html, "html.parser")
element = soup.select('div[aria-label*="star rating"]')
stars = element[0]['aria-label']

Get category.

In [243]:
element = driver.find_elements_by_xpath('//span[@class="lemon--span__373c0__3997G display--inline__373c0__3JqBP margin-r1__373c0__zyKmV border-color--default__373c0__3-ifU"]')
category = element[1].text
if '$' in category:
    category = element[2].text

Get website.

In [244]:
element = driver.find_elements_by_xpath('//a[@rel="noopener nofollow"]')
if not element:
    website = ''
else:
    website = element[0].text

Get phone number.

In [245]:
element = driver.find_elements_by_xpath('//p[contains(text(),"Phone number")]/following-sibling::p')
if not element:
    phone = ''
else:
    phone = element[0].text

Get address.

In [246]:
element = driver.find_elements_by_xpath('//a[contains(text(),"Get Directions")]/following-sibling::p')
if not element:
    address = ''
else:
    address = element[0].text

Get hours.

In [247]:
element = driver.find_elements_by_xpath('//tbody[@class="lemon--tbody__373c0__2T6Pl"]/tr')
if not hours:
    hours = []
else:
    hours = [i.text.replace('\n',' ').replace(' Closed now','') for i in element]
    hours = list(filter(None, hours))

Get number of reviews.

In [248]:
element = driver.find_elements_by_xpath('//span[@class="lemon--span__373c0__3997G text__373c0__2Kxyz text-color--white__373c0__22aE8 text-align--left__373c0__2XGa- text-weight--semibold__373c0__2l0fe text-size--large__373c0__3t60B"]')
reviews = element[0].text

Compile and get specific information.

In [249]:
df = pd.DataFrame(data={'Name':[name],'Stars':[stars],'Category':[category],'Website':[website],'Phone number':[phone],'Address':[address],'Hours':[hours],'Reviews':[reviews]})
df

,Name,Stars,Category,Website,Phone number,Address,Hours,Reviews
0,WaBa Grill,3.5 star rating,"Fast Food, Asian Fusion",,(323) 675-1090,"6316 N Figueroa St, Los Angeles, CA 90042","[Mon 11:00 AM - 9:00 PM, Tue 11:00 AM - 9:00 P...",62 reviews


In [250]:
column = input('Get value for column: ')

Get value for column: Name


In [251]:
df[column][0]

'WaBa Grill'

### Scrape Reviews

Get names.

In [252]:
def NAMES(driver):
    element = driver.find_elements_by_xpath('//div[@class="lemon--div__373c0__1mboc user-passport-info border-color--default__373c0__3-ifU"]/span/a')
    name = [i.text for i in element]
    return name

Get star ratings.

In [253]:
def STARS(driver):
    element = driver.find_elements_by_xpath('//div[@class="lemon--div__373c0__1mboc review__373c0__13kpL border-color--default__373c0__3-ifU"]/div/div/div/span/div')
    stars = [int(i.get_attribute('aria-label').replace(' star rating','')) for i in element]
    return stars

Get date.

In [254]:
def DATE(driver):
    element = driver.find_elements_by_xpath('//div[@class="lemon--div__373c0__1mboc review__373c0__13kpL border-color--default__373c0__3-ifU"]/div/div/div/span')
    date = [i.text for i in element if '/' in i.text]
    return date

Get number "Useful".

In [255]:
def USEFUL(driver):
    element = driver.find_elements_by_xpath('//span[contains(text(),"Useful")]')
    useful = [i.text.replace('Useful','').replace(' ','') for i in element]
    useful = [0 if i=='' else int(i) for i in useful]
    return useful

Get number "Funny".

In [256]:
def FUNNY(driver):
    element = driver.find_elements_by_xpath('//span[contains(text(),"Funny")]')
    funny = [i.text.replace('Useful','').replace(' ','') for i in element]
    funny = [0 if i=='' else int(i) for i in useful]
    return funny

Get number "Cool".

In [257]:
def COOL(driver):
    element = driver.find_elements_by_xpath('//span[contains(text(),"Cool")]')
    cool = [i.text.replace('Useful','').replace(' ','') for i in element]
    cool = [0 if i=='' else int(i) for i in useful]
    return cool

Get text of reviews.

In [258]:
def TEXT(driver):
    element = driver.find_elements_by_xpath('//div[@class="lemon--div__373c0__1mboc margin-b2__373c0__abANL border-color--default__373c0__3-ifU"]/p/span')
    text = [i.text.replace('\n',' ') for i in element]
    return text

Compile all reviews.

In [259]:
element = driver.find_elements_by_xpath('//div[@class="lemon--div__373c0__1mboc pagination__373c0__3z4d_ border--top__373c0__3gXLy border--bottom__373c0__3qNtD border-color--default__373c0__3-ifU"]')
pages = int(element[0].text[len(element[0].text)-1])

In [261]:
names = NAMES(driver)
stars = STARS(driver)
date = DATE(driver)
text = TEXT(driver)
useful = USEFUL(driver)
funny = FUNNY(driver)
cool = COOL(driver)
df = pd.DataFrame(data={'Name':names,'Stars':stars,'Date':date,'Text':text})
for i in range(pages-1):
    button = driver.find_elements_by_xpath('//div[@class="lemon--div__373c0__1mboc navigation-button-container__373c0__2BpRi border-color--default__373c0__3-ifU"]')[1]
    button.click()
    time.sleep(4)
    names = NAMES(driver)
    stars = STARS(driver)
    date = DATE(driver)
    text = TEXT(driver)
    useful = USEFUL(driver)
    funny = FUNNY(driver)
    cool = COOL(driver)
    df = df.append(pd.DataFrame(data={'Name':names,'Stars':stars,'Date':date,'Text':text,'Useful':useful,'Funny':funny,'Cool':cool})).reset_index(drop=True)
df

,Name,Stars,Date,Text,Useful,Funny,Cool
0,Vivian V.,5,8/3/2020,I come here often when I want to eat something...,NaN,NaN,NaN
1,Salomon D.,1,11/16/2020,"I know we're are in the middle of a pandemic, ...",NaN,NaN,NaN
2,Kimberly M.,1,8/19/2020,I FOUND A METAL NAIL IN MY PLATE!!!! I am stil...,NaN,NaN,NaN
3,Karen S.,5,11/25/2020,They are all humble workers. I enjoy their sa...,NaN,NaN,NaN
4,Nellie W.,2,6/2/2020,I came to pickup lunch here after seeing a cli...,NaN,NaN,NaN
...,...,...,...,...,...,...,...
57,Giovanni C.,5,8/20/2016,Great service for restaurant that has been ope...,0.0,0.0,0.0
58,Tiffany A.,1,12/16/2017,The reason why I am giving this rating is not ...,1.0,1.0,1.0
59,Eva N.,5,5/24/2017,Always great experience when I eat here. Excel...,0.0,0.0,0.0
60,Ray F.,5,2/11/2017,My experience at this Waba Grill yesterday was...,0.0,0.0,0.0
